In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

# Part 1: Importing and Exploring Data****

In [ ]:
df = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.keys()

In [ ]:
df.tail()

# Part 2: Visualizing The Data

Some basic visualizations of some of the variables

In [ ]:
sns.pairplot(df, hue = 'DEATH_EVENT', vars = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 'platelets', 'serum_creatinine', 
                                             'serum_sodium', 'time'])

In [ ]:
sns.countplot(df['DEATH_EVENT'])

In [ ]:
sns.countplot(df['smoking'])

In [ ]:
sns.countplot(df['high_blood_pressure'])

In [ ]:
sns.countplot(df['sex'])

In [ ]:
sns.countplot(df['diabetes'])

In [ ]:
plt.figure(figsize = (20,10))

sns.countplot(df['age'])

In [ ]:
sns.scatterplot(x = 'serum_sodium', y = 'serum_creatinine', hue = 'DEATH_EVENT', data = df)

In [ ]:
sns.boxplot(x = 'sex', y = 'age', data = df)

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(df.corr(), annot = True)

# Part 3: Building a Model 

For this, we're going to do  a model based on time, the medicines, platelets, and age

In [ ]:
X = df[['time', 'ejection_fraction', 'serum_creatinine', 'age']]


In [ ]:
X.head()

In [ ]:
y = df['DEATH_EVENT']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
svc_model = SVC()
svc_model.fit(X_train, y_train)

In [ ]:
y_predict = svc_model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_predict)

In [ ]:
sns.heatmap(cm, annot = True)

In [ ]:
min_train = X_train.min()
range_train = (X_train-min_train).max()
X_train_scaled = (X_train - min_train)/range_train

In [ ]:
sns.scatterplot(x = X_train['age'], y = X_train['time'], hue = y_train)

In [ ]:
min_test = X_test.min()
range_test = (X_test - min_test).max()
X_test_scaled = (X_test - min_test)/range_test

In [ ]:
svc_model.fit(X_train_scaled, y_train)

In [ ]:
y_predict = svc_model.predict(X_test_scaled)

In [ ]:
cm = confusion_matrix(y_test, y_predict)

In [ ]:
sns.heatmap(cm, annot=True)

In [ ]:
print(classification_report(y_test, y_predict))

In [ ]:
param_grid = {'C' : [0.1, 1, 10, 100], 'gamma' : [1, .1, .01, .001], 'kernel' : ['rbf']}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 4)

In [ ]:
grid.fit(X_train_scaled, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid_predictions = grid.predict(X_test_scaled)

In [ ]:
cm = confusion_matrix(y_test, grid_predictions)

In [ ]:
sns.heatmap(cm, annot=True)

In [ ]:
print(classification_report(y_test, grid_predictions))

# Part 4: Building A Logistic Regression Model
Building a Logistic Regression Model based on diabetes, smoking, and high blood pressure

In [ ]:
X = df[['time', 'ejection_fraction', 'serum_creatinine', 'age', 'high_blood_pressure']]

y = df['DEATH_EVENT']

from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
logmodel = LogisticRegression()

In [ ]:
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

# PART 5: KNN Model


In [ ]:
from sklearn.model_selection import train_test_split
X = df[['time', 'ejection_fraction', 'serum_creatinine', 'age']]
y = df['DEATH_EVENT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)

In [ ]:
pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))

In [ ]:
print(classification_report(y_test,pred))

In [ ]:
error_rate = []

# Will take some time
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
knn = KNeighborsClassifier(n_neighbors=10)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=10')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))